In [6]:
# from datasets import load_dataset
# load_dataset('json', data_files='personal.json', keep_in_memory=True)

In [1]:
# from transformers import Trainer, TrainingArguments, GPT2Tokenizer, GPT2LMHeadModel
# from datasets import load_dataset

# # Load your dataset
# dataset = load_dataset('json', data_files='personal.json')
# # dataset = load_dataset('json', data_files='personal.json', keep_in_memory=True)


# # Split the dataset into training and evaluation sets (90% train, 10% eval)
# # train_test_split = dataset['train'].train_test_split(test_size=0.1)
# train_test_split = dataset['train'].train_test_split(test_size=0.1, load_from_cache_file=False)
# train_dataset = train_test_split['train']
# eval_dataset = train_test_split['test']

# # Load the tokenizer and model
# model_name = 'distilgpt2'  # Use a smaller model
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)
# # model = GPT2LMHeadModel.from_pretrained("./results")  # load your already fine-tuned model

# # Add a padding token if it doesn't exist
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

# # Tokenize the datasets
# def tokenize_function(examples):
#     # Ensure keys match your JSON structure, e.g., "input" and "output"
#     model_inputs = tokenizer(
#         examples["input"],
#         padding="max_length",
#         truncation=True,
#         max_length=128
#     )
#     labels = tokenizer(
#         examples["output"],
#         padding="max_length",
#         truncation=True,
#         max_length=128
#     )["input_ids"]
    
#     model_inputs["labels"] = labels
#     return model_inputs

# # Apply tokenization
# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# # Set training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="steps",
#     learning_rate=5e-5,
#     per_device_train_batch_size=1,
#     # per_device_eval_batch_size=1,
#     num_train_epochs=5,
#     weight_decay=0.05,
#     # save_strategy="epoch",
#     # save_total_limit=2,
#     # load_best_model_at_end=True,
#     # report_to="none"
#     logging_dir='./logs',
#     logging_steps=10
# )



# # Define the trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_eval_dataset,
#     tokenizer=tokenizer,
# )

# # Train the model
# trainer.train()

# # Save the tokenizer and model after training
# tokenizer.save_pretrained("./results")
# model.save_pretrained("./results", safe_serialization=False)



In [3]:
# from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

# model_name = "facebook/blenderbot-400M-distill"
# tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
# model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# personal_info = {
#     "what is your name": "Fahad Amin",
#     "who are you": "Fahad Amin",
#     "what is your job?": "Lecturer in AI",
#     "where do you live": "Sahiwal",
#     "which city are you from": "Sahiwal"
# }

# while True:
#     user_input = input("You: ").strip().lower()
#     found = False

#     for question, answer in personal_info.items():
#         if question in user_input:
#             print("Bot:", answer)
#             found = True
#             break

#     if not found:
#         inputs = tokenizer([user_input], return_tensors="pt")
#         reply_ids = model.generate(**inputs)
#         reply = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
#         print("Bot:", reply)


In [4]:
# import json

# data = [
#     {"input": "What is your name?", "output": "My name is Fahad Amin Ramay."},
#     {"input": "Where do you work?", "output": "I am a lecturer at COMSATS University Sahiwal."},
#     {"input": "Tell me about yourself.", "output": "My name is Fahad Amin Ramay, and I teach at COMSATS University Sahiwal."},
#     {"input": "What do you like?", "output": "I like programming."},
#     {"input": "What subjects do you teach?", "output": "I teach AI, Machine Learning, Operating Systems, Programming, Computer Vision."},
#     {"input": "Which department are you in?", "output": "I am in the Faculty of Software Engineering."},
#     {"input": "Which is the biggest department at your university?", "output": "The Computer Science department is the biggest department."}
# ]

# filename = "personal.json"
# with open(filename, 'w') as json_file:
#     json.dump(data, json_file, indent=4)

# print(f"Data has been written to {filename}")


Data has been written to personal.json


In [6]:
from transformers import Trainer, TrainingArguments, GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

# Load dataset
dataset = load_dataset('json', data_files='personal.json')
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Load tokenizer and model
model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'
model.resize_token_embeddings(len(tokenizer))

# Tokenize
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input'], padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output'], padding='max_length', truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.05,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

# Train
trainer.train()

# Save model and tokenizer
tokenizer.save_pretrained('./results')
model.save_pretrained('./results')


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

C:\Users\Hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
10,12.195900,11.306578
20,10.989400,10.643401
30,10.495400,10.202866
40,10.103400,9.773734
50,9.737500,9.418766
60,9.519600,9.264100


C:\Users\Hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [18]:
# test your bot
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = './results'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

def generate_response(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_p=0.95,
            top_k=50,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example
prompt = "What is your name?"
response = generate_response(prompt)
print(f"Prompt: {prompt}\nResponse: {response}")


The following generation flags are not valid and may be ignored: ['top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Prompt: What is your name?
Response: What is your name?

The following is a transcript of the conversation between the two men.
This transcript has been automatically generated and may not be 100% accurate.


In [11]:
# print(f"Sample input: {tokenized_train_dataset[0]['input_ids']}")
# print(f"Sample output: {tokenized_train_dataset[0]['labels']}")